RNN을 해보자

In [3]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()

tf.compat.v1.reset_default_graph()   # RNN keep weights in the memory

idx2char=['h','i','e','l','o']
x_data=[[0,1,0,2,3,3]]   #'hi hell', 입력값과 출력값개수를 맞추기위해서 마지막 인덱스 제외
y_data=[[1,0,2,3,3,4]]   # i h e l l o       => h i h e l l -> i h e l l o

#one_hot encoding
x_one_hot=[[[1,0,0,0,0],
            [0,1,0,0,0],
            [1,0,0,0,0],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [0,0,0,1,0]]]    # [0,1,0,2,3,3]을 one_hot encoding한 결과
#print(np.array(enc.fit(x_data)))


input_dim=5   # x_one_hot에서 axis=2의 차원 수
batch_size=1  # 예를들어, 예측할 단어가 hihell friend에서 hihell
sequence_length=6  # n개의 sequence를 가진 데이터를 이용해 n개의 sequence(셀의 개수)를 예측하도록 그 n개 = sequence_length
num_class=5

learning_rate=0.1

# RNN
X=tf.placeholder(tf.float32, [None,sequence_length, input_dim])
Y=tf.placeholder(tf.int32, [None,sequence_length])  # index를 맞추는 모델 생성
cell=tf.contrib.rnn.BasicLSTMCell(num_units=5, state_is_tuple=True) 
initial_state=cell.zero_state(batch_size, tf.float32)  # 처음에 셀에 들어가는 초기값은 0으로 설정 
output, _staste=tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
output_shape=output.shape

# Fully Connected
x_for_fc = tf.reshape(output, [-1,5])  # num_units=5이기때문에, [None,5]로 reshape해줌
outputs=tf.contrib.layers.fully_connected(inputs=x_for_fc,
                                              num_outputs=num_class,
                                          activation_fn=None)  # 선형회귀(activation_fn=None)로 학습

outputs=tf.reshape(outputs,[batch_size, sequence_length, num_class])

weight=tf.ones([batch_size, sequence_length])  #weight=[1,1,1,1,1,1]으로 설정
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,  # seq2seq : sequence(outputs)와 sequence(Y) 사이의 거리(차)를 계산해줌
                                               targets=Y,       # Y의 차원이 logits차원을 기준으로 one_hot_encoding 됨(seq2seq).
                                               weights=weight)  # weights= weight(1로된 벡터)  -> 1 * 거리 + 1*거리... = 값 

loss=tf.reduce_mean(sequence_loss)  
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction=tf.argmax(outputs,axis=2)


with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss,train], feed_dict={X : x_one_hot, Y : y_data})
        result=sess.run(prediction, feed_dict={X : x_one_hot})
        print('loss : ',l, 'prediction : ',result,'Y : ',y_data)

        result_str=[idx2char[c] for c in np.squeeze(result)]  # np.squeeze() -> [5,1,5] -> [5,5] (차원수가 1인 차원 제거 )
        print('\tPrediction : ',''.join(result_str))
    
    output_shape=sess.run()
    print(output_shape)


AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [53]:
from tensorflow.examples.tutorials.mnist import input_data
import cifar_cnn
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()

d = cifar_cnn.CifarDataManager()
images = d.train.images

#cifar_cnn.display_cifar(images, 10)
print(d.train.images.shape)  # 50000, 32, 32, 3
print(d.train.labels.shape)  #50000, 10

tf.compat.v1.reset_default_graph()   # RNN keep weights in the memory

input_dim=10   # x_one_hot에서 axis=2의 차원 수
batch_size=32  
sequence_length=32  # n개의 sequence를 가진 데이터를 이용해 n개의 sequence(셀의 개수)를 예측하도록 그 n개 = sequence_length
num_class=10

learning_rate=0.1

x_data=d.train.images
y_data=d.train.labels

X = tf.placeholder(tf.float32, [None, 32,32,3])  # N,sequence_length, sequence_length,3
Y = tf.placeholder(tf.float32, [None,10])      
X_img=tf.reshape(X,[-1,32*32*1])

idx2char=[]

cell=tf.contrib.rnn.BasicLSTMCell(num_units=10, state_is_tuple=True) 
initial_state=cell.zero_state(batch_size, tf.float32)  # 처음에 셀에 들어가는 초기값은 0으로 설정 
output, _staste=tf.nn.dynamic_rnn(cell, X_img, initial_state=initial_state, dtype=tf.float32)

# Fully Connected
x_for_fc = tf.reshape(output, [-1,10])  # num_units=5이기때문에, [None,5]로 reshape해줌
outputs=tf.contrib.layers.fully_connected(inputs=x_for_fc,
                                          num_outputs=num_class,
                                          activation_fn=None)  # 선형회귀(activation_fn=None)로 학습

outputs=tf.reshape(outputs,[batch_size, sequence_length, num_class])

weight=tf.ones([batch_size, sequence_length])  #weight=[1,1,1,1,1,1]으로 설정
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,  # seq2seq : sequence(outputs)와 sequence(Y) 사이의 거리(차)를 계산해줌
                                               targets=Y,       # Y의 차원이 logits차원을 기준으로 one_hot_encoding 됨(seq2seq).
                                               weights=weight)  # weights= weight(1로된 벡터)  -> 1 * 거리 + 1*거리... = 값 

loss=tf.reduce_mean(sequence_loss)  
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction=tf.argmax(outputs,axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    '''
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(d.train.images.shape[0] / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = d.train.next_batch(batch_size)
            feed_dict = {X_img:batch_xs, Y:batch_ys, keep_prob:0.7}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch
            
        print(f'Epoch : {epoch+1}, Cost : {avg_cost}')
        
    print("Learning finished")
    '''
    for i in range(50):
        l, _ = sess.run([loss,train], feed_dict={X : x_data, Y : y_data})
        result=sess.run(prediction, feed_dict={X : x_data})
        print('loss : ',l, 'prediction : ',result,'Y : ',y_data)

        result_str=[idx2char[c] for c in np.squeeze(result)]  # np.squeeze() -> [5,1,5] -> [5,5] (차원수가 1인 차원 제거 )
        print('\tPrediction : '.join(result_str))


'''
x_data=
x_data=np.array(x_data).reshape(-1,10)
enc.fit(x_data)
x_one_hot=enc.transform(x_data).toarray()
x_one_hot
#x_data=tf.reshape(x_data,[,-1,32,1])
#y_data=[0,0,1,0,0,0,0,0,0]   # 숫자는 2

'''





(50000, 32, 32, 3)
(50000, 10)


ValueError: Shape (1024, ?) must have rank at least 3

원핫 인코딩하는 방법

In [50]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()

tf.compat.v1.reset_default_graph()   # RNN keep weights in the memory

idx2char=['h','i','e','l','o']
x_data=[[0,1,0,2,3,3]]   #'hi hell', 입력값과 출력값개수를 맞추기위해서 마지막 인덱스 제외
y_data=[[1,0,2,3,3,4]]   # i h e l l o       => h i h e l l -> i h e l l o

#one_hot encoding
#x_one_hot=[[[1,0,0,0,0],
#            [0,1,0,0,0],
#            [1,0,0,0,0],
#            [0,0,1,0,0],
#            [0,0,0,1,0],
#            [0,0,0,1,0]]]    # [0,1,0,2,3,3]을 one_hot encoding한 결과
x_data=np.array(x_data).reshape(-1,10)
enc.fit(x_data)
x_one_hot=enc.transform(x_data).toarray()
x_one_hot

print(d.train.labels)
print(d.train.labels.shape) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
(50000, 10)


In [56]:
mnist = input_data.read_data_sets("./samples/MNIST_data/", one_hot=True)


Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz
(3,)
